# Below are some parts of the code I used in Iman Company to develope credit risk models using python and sql

## Import data from PostgreSQL + Example of SQL code I used

In [ ]:
def fetch_query (query):
    
    with psycopg2.connect(host = '...', port = '...',
                      dbname = '...', user = '...', password = '...') as con:
        with con.cursor() as cursor:
            cursor.execute(query)
            fetched_result = cursor.fetchall()
            
            # Extract the column names
            col_names = []
            for elt in cursor.description:
                col_names.append(elt[0])
    
    return fetched_result, col_names

df, col_names = fetch_query("""
with max_overdue_days_tbl as
(select ib.installment_id,
    max(overdue_day) max_overdue_day,
    max(overdue_day) filter(where ib.period_number = 1) max_overdue_day_fpd
from installment_service_installment_bills ib
join installment_service_installment i on ib.installment_id = i.guid
where status != 'canceled'
group by 1
having max(overdue_day)>0)

select count(installment_id) over(partition by created_at::date) daily_sales,
installment_id,
phone_number, 
bpm_p_date_start::date scoring_date, 
created_at::date purchased_date,
coalesce(registration_adress_passport, registration_adress_cb),
 (CASE WHEN max_overdue_day >90
       THEN 1
       WHEN max_overdue_day <=90
       THEN 0
       ELSE Null
  END) as npl90_flag,
 (CASE WHEN max_overdue_day >60
       THEN 1
       WHEN max_overdue_day <=60
       THEN 0
       ELSE Null
  END) as npl60_flag,
from scoring_tbl
join max_overdue_days_tbl using(installment_id)
""")
data = pd.DataFrame(df, columns=col_names)

## Checking npl90+ (bad clients) depending on credit history, income level and age
Проверка npl90+ (bad clients) в зависимости от наличия кредитной истории, уровня дохода и возраста

In [ ]:
# с помощью сводных таблиц
round(pd.crosstab([data.fn_cb_has_crd_hst], [data.inc_cat, data.age_cat], values = data.bad, aggfunc='mean') * 100)

## Box-and-whisker plot for downpayment percentage displaying

In [ ]:
fig= plt.figure(figsize =(12, 2))
plt.boxplot(data_del.query('downpayment>0')['downpayment']
            / data_del.query('downpayment>0')['installment_amount'] * 100, vert=False)

plt.title("Величина downpayment в процентах от величины рассрочки, в %")
plt.xlim(0, 8)
plt.grid(True)

plt.show()

## Checking the correlation between client and installment (credit) parameters (phik_matrix)
Проверка корреляции между параметрами клиента и рассрочки (phik_matrix)

In [ ]:
# матрица корреляции
interval_cols = ['installment_amount', 'fn_cb_del_last_3y_max_buc',  'kred_nagruzka', 'age', 'final_income']

corr_matrix = data[['installment_amount', 'bad_or_good_client_by_fpd_90dpd', 
                    'fn_cb_del_last_3y_max_buc', 'fn_cb_has_crd_hst', 'kred_nagruzka', 
                    'age', 'final_income']].phik_matrix(interval_cols=interval_cols)

plot_correlation_matrix(corr_matrix.round(2).values, 
                        x_labels=corr_matrix.round(2).columns, 
                        y_labels=corr_matrix.round(2).index, 
                        vmin=0, vmax=1, color_map="Greens", 
                        title=r"correlation $\phi_K$", 
                        fontsize_factor=1.5, 
                        figsize=(14, 8))
plt.tight_layout()

## Checking correlation using the significance matrix
Проверка корреляции по матрице значимости (significance_matrix)

In [ ]:
# построим матрицу значимости, чтобы проверить значима ли корреляци в таблице выше
significance_overview = data_ml.significance_matrix(interval_cols=interval_cols)
plot_correlation_matrix(significance_overview.fillna(0).values, 
                        x_labels=significance_overview.columns, 
                        y_labels=significance_overview.index, 
                        vmin=-5, vmax=5, title="Significance of the coefficients", 
                        usetex=False, fontsize_factor=1.5, figsize=(16, 10))
plt.tight_layout()

## Training ML model CatBoostClassifier
Обучение ML модели CatBoostClassifier

In [ ]:
# создадим функцию для иттераций
model=[]
cat_features = ['fn_cb_has_crd_hst', 'gender', 'max_delay_3y_90d_flag', ....]

def objective(trial):
    model = CatBoostClassifier(random_state=130623, 
        iterations=trial.suggest_int("iterations", 100, 300),
        learning_rate = trial.suggest_categorical('learning_rate', [0.005, 0.02, 0.05, 0.08, 0.1]),
        depth=trial.suggest_int("depth", 4, 10),
        verbose=False,
        cat_features=cat_features)
    scores = cross_val_score(model, features_train_cat, target_train, scoring='roc_auc', cv=5)
    final_score = scores.mean()
    return final_score

# обучим модель
study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 10)
print('Наилучший precision_score для модели CatBoostClassifier:', study.best_value)

## Testing ML model CatBoostClassifier
Тестирование ML модели CatBoostClassifier

In [ ]:
cat_features = category_columns
model = CatBoostClassifier(random_state=130623, 
                           iterations=127, 
                           learning_rate = 0.08, 
                           verbose=False, 
                           depth=4,
                           cat_features=cat_features)
model.fit(features_train_cat, target_train)
predictions = model.predict(features_test_cat)
predictions_proba = model.predict_proba(features_test_cat)

print('AUC ROC модели  CatBoostClassifier на тестовой выборке:', 
      roc_auc_score(target_test, predictions_proba[:,1]))

# построим матрицу ошибок
pd.DataFrame(confusion_matrix( target_test, predictions),
             columns = ['Forecast 0', 'Forecast 1'],
             index = ['Actual 0', 'Actual 1'])